### 查看视频分辨率

In [9]:
import cv2
import os # 引入os模块来检查文件是否存在

def get_video_resolution_cv2(video_path):
    """
    使用 OpenCV 获取视频的分辨率。
    :param video_path: 视频文件的路径
    :return: (宽度, 高度) 元组，如果失败则返回 None
    """
    if not os.path.exists(video_path):
        print(f"❌ 错误：文件不存在 -> {video_path}")
        return None

    try:
        # 打开视频文件
        vid_capture = cv2.VideoCapture(video_path)

        if not vid_capture.isOpened():
            print(f"❌ 错误：无法打开视频文件 -> {video_path}")
            return None

        # 获取帧宽度和高度
        width = int(vid_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(vid_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

        # 释放资源
        vid_capture.release()

        return (width, height)

    except Exception as e:
        print(f"处理视频时发生未知错误: {e}")
        return None

# --- 使用示例 ---
# ⚠️ 请将 'your_video.mp4' 替换为你的视频文件实际路径
video_file = '/home/darwin/projects/myNootbook/ToolsBook/resources/202510152137(1).mp4' 
resolution = get_video_resolution_cv2(video_file)

if resolution:
    print(f"✅ 视频分辨率是: {resolution[0]}x{resolution[1]}")

✅ 视频分辨率是: 1920x1080


### 降低视频分辨率

In [12]:
import cv2
import os

def reduce_video_resolution(input_path, output_path, new_width):
    """
    降低视频文件的分辨率并保持宽高比。

    参数:
    input_path (str): 输入视频文件的路径。
    output_path (str): 输出视频文件的路径。
    new_width (int): 视频的新宽度，高度将自动计算以保持宽高比。
    """
    # 1. 打开视频文件
    cap = cv2.VideoCapture(input_path)

    # 检查视频是否成功打开
    if not cap.isOpened():
        print(f"❌ 错误: 无法打开视频文件 {input_path}")
        return

    # 2. 获取原始视频的属性
    original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # 如果新宽度大于等于原始宽度，就没必要处理了
    if new_width >= original_width:
        print(f"ℹ️ 提示: 目标宽度 ({new_width}px) 大于或等于原始宽度 ({original_width}px)。无需降低分辨率。")
        cap.release()
        return

    # 3. 计算新的高度以保持宽高比
    aspect_ratio = original_height / original_width
    new_height = int(new_width * aspect_ratio)
    new_resolution = (new_width, new_height)

    print(f"原始分辨率: {original_width}x{original_height}")
    print(f"新的分辨率: {new_width}x{new_height}")
    print(f"帧率 (FPS): {fps}")

    # 4. 定义视频编码器和创建 VideoWriter 对象
    # 对于 .mp4 文件，'mp4v' 或 'avc1' 是不错的选择
    # 对于 .avi 文件，可以尝试 'XVID'
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
    out = cv2.VideoWriter(output_path, fourcc, fps, new_resolution)

    # 5. 逐帧读取、调整大小并写入新文件
    while True:
        # ret 是一个布尔值，如果成功读取帧则为 True
        # frame 是读取到的图像帧
        ret, frame = cap.read()

        if not ret:
            break # 视频读取完毕或发生错误

        # 🖼️ 核心步骤：调整帧的大小
        # cv2.INTER_AREA 插值方法通常是缩小图像的最佳选择
        resized_frame = cv2.resize(frame, new_resolution, interpolation=cv2.INTER_AREA)

        # 🎬 将调整后的帧写入输出文件
        out.write(resized_frame)

    # 6. 释放所有资源
    print(f"\n🎉 视频处理完成，已保存至 {output_path}")
    cap.release()
    out.release()
    # cv2.destroyAllWindows()

# --- 使用示例 ---
if __name__ == '__main__':
    # 替换为你的视频文件路径
    input_video_path = '/home/darwin/projects/myNootbook/ToolsBook/resources/202510152137(1).mp4' 
    output_video_path = '/home/darwin/projects/myNootbook/ToolsBook/resources/202510152137_reduced.mp4'

    # 设置你想要的目标宽度（例如，640像素）
    target_width = 1080 

    # 检查输入文件是否存在
    if not os.path.exists(input_video_path):
        print(f"❌ 错误: 输入文件 '{input_video_path}' 不存在。请检查文件路径是否正确。")
    else:
        reduce_video_resolution(input_video_path, output_video_path, target_width)

原始分辨率: 1920x1080
新的分辨率: 1080x607
帧率 (FPS): 30.089285714285715

🎉 视频处理完成，已保存至 /home/darwin/projects/myNootbook/ToolsBook/resources/202510152137_reduced.mp4


### 视频提取成帧

In [13]:
import cv2
import os

def extract_frames(video_path, output_folder, frame_interval=1):
    """
    从视频中提取帧并保存为图片。

    :param video_path: 视频文件的路径。
    :param output_folder: 保存帧的输出文件夹路径。
    :param frame_interval: 帧提取间隔。默认为 1，表示提取每一帧。
                           例如，设置为 5 表示每隔 5 帧提取一帧。
    """
    # 1. 检查视频路径是否存在
    if not os.path.exists(video_path):
        print(f"❌ 错误：找不到视频文件 '{video_path}'")
        return

    # 2. 确保输出文件夹存在，如果不存在则创建
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f"📁 已创建文件夹: '{output_folder}'")

    # 3. 打开视频文件
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"❌ 错误：无法打开视频文件 '{video_path}'")
        return

    # 4. 循环读取视频帧
    frame_count = 0
    saved_frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # 5. 根据设定的间隔保存帧
        if frame_count % frame_interval == 0:
            # ✨ --- 核心修改点在这里 --- ✨
            # 将文件名格式改为两位数，并用0填充，例如：01.png, 02.png ... 10.png
            image_name = f"{(saved_frame_count + 1):02d}.png"
            # ✨ --- 核心修改点结束 --- ✨
            
            output_path = os.path.join(output_folder, image_name)
            
            cv2.imwrite(output_path, frame)
            print(f"✅ 已保存: {output_path}")
            saved_frame_count += 1

        frame_count += 1

    # 6. 释放资源
    cap.release()
    print(f"\n🎉 抽帧完成！总共从视频中读取了 {frame_count} 帧，保存了 {saved_frame_count} 张图片。")


# --- 如何使用 ---
if __name__ == "__main__":
    video_file = '/home/darwin/projects/myNootbook/ToolsBook/resources/202510152137_reduced.mp4'
    output_dir = '/home/darwin/projects/myNootbook/ToolsBook/output/202510152137_reduced'  # 建议修改输出文件夹名以区分
    
    extract_frames(video_file, output_dir)

📁 已创建文件夹: '/home/darwin/projects/myNootbook/ToolsBook/output/202510152137_reduced'
✅ 已保存: /home/darwin/projects/myNootbook/ToolsBook/output/202510152137_reduced/01.png
✅ 已保存: /home/darwin/projects/myNootbook/ToolsBook/output/202510152137_reduced/02.png
✅ 已保存: /home/darwin/projects/myNootbook/ToolsBook/output/202510152137_reduced/03.png
✅ 已保存: /home/darwin/projects/myNootbook/ToolsBook/output/202510152137_reduced/04.png
✅ 已保存: /home/darwin/projects/myNootbook/ToolsBook/output/202510152137_reduced/05.png
✅ 已保存: /home/darwin/projects/myNootbook/ToolsBook/output/202510152137_reduced/06.png
✅ 已保存: /home/darwin/projects/myNootbook/ToolsBook/output/202510152137_reduced/07.png
✅ 已保存: /home/darwin/projects/myNootbook/ToolsBook/output/202510152137_reduced/08.png
✅ 已保存: /home/darwin/projects/myNootbook/ToolsBook/output/202510152137_reduced/09.png
✅ 已保存: /home/darwin/projects/myNootbook/ToolsBook/output/202510152137_reduced/10.png
✅ 已保存: /home/darwin/projects/myNootbook/ToolsBook/output/2025101521

### 视频提取帧——低分辨率

In [1]:
import cv2
import os

def extract_frames(video_path, output_folder, frame_interval=1, scale_factor=1.0):
    """
    从视频中提取帧，调整分辨率后保存为图片。

    :param video_path: 视频文件的路径。
    :param output_folder: 保存帧的输出文件夹路径。
    :param frame_interval: 帧提取间隔。默认为 1。
    :param scale_factor: 缩放比例。默认为 1.0 (原始大小)。
                         例如，0.5 表示将宽高都缩放到 50%。
    """
    # 1. 检查视频路径是否存在
    if not os.path.exists(video_path):
        print(f"❌ 错误：找不到视频文件 '{video_path}'")
        return

    # 2. 确保输出文件夹存在
    os.makedirs(output_folder, exist_ok=True)
    if not os.listdir(output_folder): # 仅在文件夹为空时打印创建信息
        print(f"📁 已创建/指定文件夹: '{output_folder}'")

    # 3. 打开视频文件
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"❌ 错误：无法打开视频文件 '{video_path}'")
        return

    # 4. 循环读取视频帧
    frame_count = 0
    saved_frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_interval == 0:
            # --- 缩放图像的核心代码 ---
            if scale_factor != 1.0:
                # 如果设置了缩放比例，就调整图像大小
                new_width = int(frame.shape[1] * scale_factor)
                new_height = int(frame.shape[0] * scale_factor)
                dim = (new_width, new_height)
                
                # cv2.resize 进行缩放，INTER_AREA 算法适合缩小图像
                resized_frame = cv2.resize(frame, dim, interpolation=cv2.INTER_AREA)
            else:
                # 如果不缩放，则使用原始帧
                resized_frame = frame
            # --- 核心代码结束 ---

            # ✨ --- 核心修改点在这里 --- ✨
            # 将文件名格式改为两位数，并用0填充，同时后缀改为 .png
            image_name = f"{(saved_frame_count + 1):02d}.png"
            output_path = os.path.join(output_folder, image_name)
            
            # 保存的是缩放后的帧 (resized_frame)
            cv2.imwrite(output_path, resized_frame)
            print(f"✅ 已保存 (尺寸: {resized_frame.shape[1]}x{resized_frame.shape[0]}): {output_path}")
            saved_frame_count += 1

        frame_count += 1

    # 6. 释放资源
    cap.release()
    print(f"\n🎉 抽帧完成！总共从视频中读取了 {frame_count} 帧，保存了 {saved_frame_count} 张图片。")


# --- 如何使用 ---
if __name__ == "__main__":
    video_file = "/home/darwin/projects/myNootbook/ToolsBook/resources/矿下纹理.mp4"       # <-- 修改这里
    output_dir = "/home/darwin/projects/myNootbook/ToolsBook/output/矿下纹理"  # <-- 建议修改文件夹名

    # 💡 要将 1920*1080 的视频分辨率降低一半 (变为 960*540)
    # 只需要将 scale_factor 设置为 0.5
    scaling = 1
    
    extract_frames(video_file, output_dir, scale_factor=scaling)

📁 已创建/指定文件夹: '/home/darwin/projects/myNootbook/ToolsBook/output/矿下纹理'
✅ 已保存 (尺寸: 1080x1080): /home/darwin/projects/myNootbook/ToolsBook/output/矿下纹理/01.png
✅ 已保存 (尺寸: 1080x1080): /home/darwin/projects/myNootbook/ToolsBook/output/矿下纹理/02.png
✅ 已保存 (尺寸: 1080x1080): /home/darwin/projects/myNootbook/ToolsBook/output/矿下纹理/03.png
✅ 已保存 (尺寸: 1080x1080): /home/darwin/projects/myNootbook/ToolsBook/output/矿下纹理/04.png
✅ 已保存 (尺寸: 1080x1080): /home/darwin/projects/myNootbook/ToolsBook/output/矿下纹理/05.png
✅ 已保存 (尺寸: 1080x1080): /home/darwin/projects/myNootbook/ToolsBook/output/矿下纹理/06.png
✅ 已保存 (尺寸: 1080x1080): /home/darwin/projects/myNootbook/ToolsBook/output/矿下纹理/07.png
✅ 已保存 (尺寸: 1080x1080): /home/darwin/projects/myNootbook/ToolsBook/output/矿下纹理/08.png
✅ 已保存 (尺寸: 1080x1080): /home/darwin/projects/myNootbook/ToolsBook/output/矿下纹理/09.png
✅ 已保存 (尺寸: 1080x1080): /home/darwin/projects/myNootbook/ToolsBook/output/矿下纹理/10.png
✅ 已保存 (尺寸: 1080x1080): /home/darwin/projects/myNootbook/ToolsBook/output/矿下纹理/11

### 图片合成视频

In [15]:
import cv2
import os

def create_video_from_images(image_folder, output_video_path, fps=30):
    """
    将文件夹中的图片合成为视频。

    :param image_folder: 包含图片的文件夹路径。
    :param output_video_path: 输出视频的完整路径（例如 'output.mp4'）。
    :param fps: 视频的帧率 (Frames Per Second)。
    """
    # 1. 获取文件夹中所有的图片文件
    images = [img for img in os.listdir(image_folder) if img.endswith((".png", ".jpg", ".jpeg"))]
    
    # 关键步骤：对图片进行排序，确保视频帧的顺序正确！
    # 如果您的文件名是 01.png, 02.png, 10.png 这样，默认排序就正确。
    images.sort()

    if not images:
        print(f"❌ 错误：在文件夹 '{image_folder}' 中没有找到任何图片文件。")
        return

    print(f"🖼️ 找到了 {len(images)} 张图片。开始合成视频...")

    # 2. 读取第一张图片来获取视频的宽度和高度
    first_image_path = os.path.join(image_folder, images[0])
    frame = cv2.imread(first_image_path)
    if frame is None:
        print(f"❌ 错误：无法读取第一张图片 '{first_image_path}'。")
        return
    height, width, layers = frame.shape
    size = (width, height)

    # 3. 初始化视频写入器 (VideoWriter)
    # 定义视频编码器，'mp4v' 是用于 .mp4 格式的常用编码器
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
    out = cv2.VideoWriter(output_video_path, fourcc, fps, size)

    # 4. 循环读取每张图片并写入视频
    for i, image_name in enumerate(images):
        image_path = os.path.join(image_folder, image_name)
        frame = cv2.imread(image_path)
        
        # 确保读取到的帧和初始尺寸一致
        if frame.shape[1] == width and frame.shape[0] == height:
            out.write(frame)
            # 打印进度
            print(f"✅ 正在处理: {image_name} ({i+1}/{len(images)})")
        else:
            print(f"⚠️ 警告：跳过尺寸不匹配的图片 {image_name}")


    # 5. 释放资源
    out.release()
    print(f"\n🎉 视频合成完毕！已保存至: {output_video_path}")

# --- 如何使用 ---
if __name__ == "__main__":
    # ⭐️ 请在这里修改您的图片文件夹路径和输出视频信息 ⭐️
    
    # 输入的图片文件夹（就是我们之前抽帧创建的那个）
    img_folder = "/home/darwin/projects/myNootbook/ToolsBook/resources/result1"  # <-- 修改这里
    
    # 输出的视频文件名
    video_name = "result1.mp4"  # <-- 修改这里
    
    # 设置视频的帧率 (每秒播放多少张图片)
    frames_per_second = 30  # <-- 修改这里

    create_video_from_images(img_folder, video_name, fps=frames_per_second)

🖼️ 找到了 337 张图片。开始合成视频...
✅ 正在处理: 00001.png (1/337)
✅ 正在处理: 00002.png (2/337)
✅ 正在处理: 00003.png (3/337)
✅ 正在处理: 00004.png (4/337)
✅ 正在处理: 00005.png (5/337)
✅ 正在处理: 00006.png (6/337)
✅ 正在处理: 00007.png (7/337)
✅ 正在处理: 00008.png (8/337)
✅ 正在处理: 00009.png (9/337)
✅ 正在处理: 00010.png (10/337)
✅ 正在处理: 00011.png (11/337)
✅ 正在处理: 00012.png (12/337)
✅ 正在处理: 00013.png (13/337)
✅ 正在处理: 00014.png (14/337)
✅ 正在处理: 00015.png (15/337)
✅ 正在处理: 00016.png (16/337)
✅ 正在处理: 00017.png (17/337)
✅ 正在处理: 00018.png (18/337)
✅ 正在处理: 00019.png (19/337)
✅ 正在处理: 00020.png (20/337)
✅ 正在处理: 00021.png (21/337)
✅ 正在处理: 00022.png (22/337)
✅ 正在处理: 00023.png (23/337)
✅ 正在处理: 00024.png (24/337)
✅ 正在处理: 00025.png (25/337)
✅ 正在处理: 00026.png (26/337)
✅ 正在处理: 00027.png (27/337)
✅ 正在处理: 00028.png (28/337)
✅ 正在处理: 00029.png (29/337)
✅ 正在处理: 00030.png (30/337)
✅ 正在处理: 00031.png (31/337)
✅ 正在处理: 00032.png (32/337)
✅ 正在处理: 00033.png (33/337)
✅ 正在处理: 00034.png (34/337)
✅ 正在处理: 00035.png (35/337)
✅ 正在处理: 00036.png (36/337)
✅ 正在处理: 0003

In [10]:
import cv2
import os

# 设置图片文件夹路径和输出视频路径
img_folder = '/home/darwin/projects/myNootbook/ToolsBook/yangerhuifeng/yangerhuifeng'  # 图片帧所在的文件夹
video_path = 'output_video.mp4'  # 输出视频文件

# 获取所有图片文件名并排序（假定命名按顺序如 frame1.jpg, frame2.jpg ...）
img_files = [f for f in os.listdir(img_folder) if f.endswith(".jpg") or f.endswith(".png")]
img_files.sort()

# 读取第一张图片，获取尺寸
first_img = cv2.imread(os.path.join(img_folder, img_files[0]))
height, width, layers = first_img.shape

# 定义视频编码与输出对象
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = 25  # 帧率可根据需要调整
video_writer = cv2.VideoWriter(video_path, fourcc, fps, (width, height))

# 按顺序写入图片帧
for img_file in img_files:
    img_path = os.path.join(img_folder, img_file)
    frame = cv2.imread(img_path)
    video_writer.write(frame)

video_writer.release()
print("视频合成完成！")


视频合成完成！


### 图像重命名

In [3]:
import os

def batch_rename_images(folder_path, dry_run=True):
    """
    批量重命名图片文件，将文件名中的数字格式化为三位数（前面补零）。
    例如：'1.png' -> '001.png', '12.png' -> '012.png'

    :param folder_path: 包含图片的文件夹路径。
    :param dry_run: 是否为演练模式。True 只打印信息，不执行重命名。
    """
    print(f"📁 正在扫描文件夹: {folder_path}")
    if dry_run:
        print("💧 当前为演练模式 (Dry Run)，不会修改任何文件。")
    else:
        print("🔥 注意！当前为实战模式，将立即重命名文件。")

    # 获取文件夹中所有文件
    try:
        files = os.listdir(folder_path)
    except FileNotFoundError:
        print(f"❌ 错误：找不到文件夹 '{folder_path}'，请检查路径是否正确。")
        return

    # 对文件进行自然排序，方便查看日志
    # 先筛选出是数字文件名的png
    image_files = [f for f in files if f.endswith('.png') and os.path.splitext(f)[0].isdigit()]
    try:
        image_files.sort(key=lambda x: int(os.path.splitext(x)[0]))
    except ValueError:
        print("⚠️ 发现无法按数字排序的文件名，将使用默认排序。")
        image_files.sort()

    renamed_count = 0
    for filename in image_files:
        # 分离文件名和扩展名
        name_part, extension = os.path.splitext(filename)

        # 尝试将文件名部分转换为数字
        if name_part.isdigit():
            num = int(name_part)
            
            # 格式化为三位数，前面补零 (例如 1 -> '001', 12 -> '012')
            new_name_part = f"{num:03d}"
            new_filename = new_name_part + extension

            # 只有当新旧文件名不同时才进行操作
            if new_filename != filename:
                old_path = os.path.join(folder_path, filename)
                new_path = os.path.join(folder_path, new_filename)
                
                if dry_run:
                    print(f"[演练] 计划将 '{filename}' 重命名为 '{new_filename}'")
                else:
                    try:
                        os.rename(old_path, new_path)
                        print(f"✅ 成功: '{filename}' -> '{new_filename}'")
                    except Exception as e:
                        print(f"❌ 失败: 重命名 '{filename}' 时出错 - {e}")
                
                renamed_count += 1
    
    print("\n🎉 任务完成！")
    if renamed_count == 0:
        print("✨ 看起来所有文件都已符合命名规范，无需重命名。")
    else:
        print(f"总计处理了 {renamed_count} 个文件。")


# --- ⭐️ 请在这里配置 ⭐️ ---
if __name__ == "__main__":
    # 1. 修改为你的图片文件夹路径
    # Windows 示例: "C:\\Users\\YourUser\\Desktop\\MyImages"
    # Linux/Mac 示例: "/home/user/projects/MyImages"
    img_folder = "/home/darwin/projects/myNootbook/ToolsBook/resources/content/drive/MyDrive/projects/PyTorch-Image-Dehazing/results"  # <-- 修改这里！

    # 2. 安全开关：True = 只看不改 (演练), False = 直接修改文件 (实战)
    # 强烈建议先用 True 运行一次，检查无误后再改成 False
    is_dry_run = False  # <-- 先保持 True！

    # --- 开始执行 ---
    batch_rename_images(img_folder, dry_run=is_dry_run)

📁 正在扫描文件夹: /home/darwin/projects/myNootbook/ToolsBook/resources/content/drive/MyDrive/projects/PyTorch-Image-Dehazing/results
🔥 注意！当前为实战模式，将立即重命名文件。
✅ 成功: '01.png' -> '001.png'
✅ 成功: '02.png' -> '002.png'
✅ 成功: '03.png' -> '003.png'
✅ 成功: '04.png' -> '004.png'
✅ 成功: '05.png' -> '005.png'
✅ 成功: '06.png' -> '006.png'
✅ 成功: '07.png' -> '007.png'
✅ 成功: '08.png' -> '008.png'
✅ 成功: '09.png' -> '009.png'
✅ 成功: '10.png' -> '010.png'
✅ 成功: '11.png' -> '011.png'
✅ 成功: '12.png' -> '012.png'
✅ 成功: '13.png' -> '013.png'
✅ 成功: '14.png' -> '014.png'
✅ 成功: '15.png' -> '015.png'
✅ 成功: '16.png' -> '016.png'
✅ 成功: '17.png' -> '017.png'
✅ 成功: '18.png' -> '018.png'
✅ 成功: '19.png' -> '019.png'
✅ 成功: '20.png' -> '020.png'
✅ 成功: '21.png' -> '021.png'
✅ 成功: '22.png' -> '022.png'
✅ 成功: '23.png' -> '023.png'
✅ 成功: '24.png' -> '024.png'
✅ 成功: '25.png' -> '025.png'
✅ 成功: '26.png' -> '026.png'
✅ 成功: '27.png' -> '027.png'
✅ 成功: '28.png' -> '028.png'
✅ 成功: '29.png' -> '029.png'
✅ 成功: '30.png' -> '030.png'
✅ 成功: '31.p

### 图像重命名：00001-15330

In [14]:
import os

def rename_images_in_folder(folder_path):
    """
    将指定文件夹下的图片（如 1.png, 12.png）重命名为带有前导零的格式（如 00001.png, 00012.png）。
    """
    print(f"正在处理文件夹: {folder_path} ... 📂")
    
    # 获取文件夹中所有文件名
    try:
        filenames = os.listdir(folder_path)
    except FileNotFoundError:
        print(f"❌ 错误：找不到文件夹 '{folder_path}'。请检查路径是否正确。")
        return

    renamed_count = 0
    for filename in filenames:
        # 确保我们只处理 .png 文件
        if filename.lower().endswith('.png'):
            # 分离文件名和扩展名
            name_part, extension = os.path.splitext(filename)
            
            # 检查文件名部分是否为纯数字
            if name_part.isdigit():
                try:
                    # 将数字部分转换为整数，然后再格式化为5位的字符串（不足则在前面补0）
                    # 例如： '1' -> 1 -> '00001'
                    new_name_part = f"{int(name_part):05d}"
                    
                    # 构造新的文件名
                    new_filename = new_name_part + extension
                    
                    # 获取原始文件的完整路径和新文件的完整路径
                    old_file_path = os.path.join(folder_path, filename)
                    new_file_path = os.path.join(folder_path, new_filename)
                    
                    # 如果新旧文件名不同，则执行重命名
                    if old_file_path != new_file_path:
                        os.rename(old_file_path, new_file_path)
                        print(f"✅ 已重命名: '{filename}' -> '{new_filename}'")
                        renamed_count += 1
                    else:
                        print(f"ℹ️ 文件 '{filename}' 无需重命名。")

                except ValueError:
                    # 如果文件名部分虽然是数字但无法转换（例如超大数字），则跳过
                    print(f"⚠️ 跳过文件 '{filename}'：无法处理的文件名。")
            else:
                # 如果文件名不全是数字（例如 'background_01.png'），则跳过
                print(f"⏭️ 跳过非数字命名的文件: '{filename}'")
    
    if renamed_count > 0:
        print(f"\n✨ 任务完成！总共重命名了 {renamed_count} 个文件。")
    else:
        print("\n🤔 任务结束，没有文件被重命名。")


# --- 使用说明 ---
# 1. 将下面的路径替换为您自己的图片文件夹路径
# 2. 运行此脚本

# Windows 示例: r"C:\Users\YourName\Desktop\MyImages"
# macOS 或 Linux 示例: "/Users/YourName/Documents/MyPictures"

target_folder = "/home/darwin/projects/myNootbook/ToolsBook/output/202510152137_reduced"  # <--- ⚠️ 请修改这里！

# 调用函数开始执行
if target_folder == "请在这里填入你的文件夹路径":
    print("🛑 请先在脚本中设置 'target_folder' 的值为你的图片文件夹路径！")
else:
    rename_images_in_folder(target_folder)

正在处理文件夹: /home/darwin/projects/myNootbook/ToolsBook/output/202510152137_reduced ... 📂
✅ 已重命名: '253.png' -> '00253.png'
✅ 已重命名: '46.png' -> '00046.png'
✅ 已重命名: '12.png' -> '00012.png'
✅ 已重命名: '187.png' -> '00187.png'
✅ 已重命名: '311.png' -> '00311.png'
✅ 已重命名: '73.png' -> '00073.png'
✅ 已重命名: '249.png' -> '00249.png'
✅ 已重命名: '160.png' -> '00160.png'
✅ 已重命名: '140.png' -> '00140.png'
✅ 已重命名: '27.png' -> '00027.png'
✅ 已重命名: '19.png' -> '00019.png'
✅ 已重命名: '116.png' -> '00116.png'
✅ 已重命名: '166.png' -> '00166.png'
✅ 已重命名: '205.png' -> '00205.png'
✅ 已重命名: '68.png' -> '00068.png'
✅ 已重命名: '10.png' -> '00010.png'
✅ 已重命名: '125.png' -> '00125.png'
✅ 已重命名: '186.png' -> '00186.png'
✅ 已重命名: '189.png' -> '00189.png'
✅ 已重命名: '69.png' -> '00069.png'
✅ 已重命名: '259.png' -> '00259.png'
✅ 已重命名: '234.png' -> '00234.png'
✅ 已重命名: '84.png' -> '00084.png'
✅ 已重命名: '177.png' -> '00177.png'
✅ 已重命名: '278.png' -> '00278.png'
✅ 已重命名: '30.png' -> '00030.png'
✅ 已重命名: '128.png' -> '00128.png'
✅ 已重命名: '320.png' -> '00320.png'


---

In [1]:
path = '/home/darwin/projects/myNootbook/ToolsBook/output/extracted_frames_1080_606'

# 查看当前文件夹下面有多少张图片
import os

# 获取文件夹中所有文件名
files = os.listdir(path)

# 统计图片文件的数量
num_images = len(files)
print(f"当前文件夹下有 {num_images} 张图片。")


当前文件夹下有 15331 张图片。


In [2]:
# 提取指定照片
import os
import shutil

# --- ⚙️ 用户配置区 ---

# 1. 请将这里替换为你的原始图片所在的文件夹路径
#    例如: 'D:/MyVideos/frames' 或 '/Users/username/Desktop/frames'
source_folder = '/home/darwin/projects/myNootbook/ToolsBook/output/extracted_frames_1080_606'

# 2. 请将这里替换为你想要存放提取图片的文件夹路径
#    例如: 'D:/MyVideos/selected_frames' 或 '/Users/username/Desktop/selected_frames'
destination_folder = '/home/darwin/projects/myNootbook/ToolsBook/output/temp_img'

# 3. 这是需要提取的图片编号列表 (已为您填好)
file_numbers = [
    '00001', '00249', '00498', '00746', '00995', '01243', '01492', '01740',
    '01989', '02237', '02486', '02734', '02983', '03231', '03479', '03728',
    '03976', '04225', '04473', '04722', '04970', '05219', '05467', '05715',
    '05964', '06212', '06461', '06709', '06958', '07206', '07455', '07703',
    '07952', '08200', '08449', '08697', '08946', '09194', '09443', '09691',
    '09940', '10188', '10437', '10685', '10933', '11182', '11430', '11679',
    '11927', '12176', '12424', '12673', '12921', '13170', '13418', '13667',
    '13915', '14164', '14412', '14661', '14909', '15158'
]

# --- 🚀 主程序 (无需修改以下内容) ---

def extract_files():
    """主函数，用于执行文件提取操作"""
    # 检查源文件夹是否存在
    if not os.path.isdir(source_folder):
        print(f"❌ 错误：源文件夹 '{source_folder}' 不存在。请检查路径。")
        return

    # 如果目标文件夹不存在，则创建它
    os.makedirs(destination_folder, exist_ok=True)
    print(f"📂 目标文件夹 '{destination_folder}' 已准备就绪。")
    
    copied_count = 0
    not_found_count = 0
    
    print("\n🚀 开始提取文件...\n")
    
    # 遍历编号列表
    for number in file_numbers:
        # 构建完整的文件名 (例如: '00001.png') 和路径
        file_name = f"{number}.png"
        source_path = os.path.join(source_folder, file_name)
        destination_path = os.path.join(destination_folder, file_name)
        
        # 检查源文件是否存在
        if os.path.exists(source_path):
            # 复制文件，copy2会同时复制元数据(如修改时间)
            shutil.copy2(source_path, destination_path)
            print(f"✅ 成功复制: {file_name}")
            copied_count += 1
        else:
            # 如果文件不存在，则打印提示信息
            print(f"⚠️ 未找到文件: {file_name}，已跳过。")
            not_found_count += 1
            
    print("\n🎉 ----- 任务完成 ----- 🎉")
    print(f"总共成功复制了 {copied_count} 个文件。")
    if not_found_count > 0:
        print(f"有 {not_found_count} 个文件未在源文件夹中找到。")

if __name__ == "__main__":
    extract_files()

📂 目标文件夹 '/home/darwin/projects/myNootbook/ToolsBook/output/temp_img' 已准备就绪。

🚀 开始提取文件...

✅ 成功复制: 00001.png
✅ 成功复制: 00249.png
✅ 成功复制: 00498.png
✅ 成功复制: 00746.png
✅ 成功复制: 00995.png
✅ 成功复制: 01243.png
✅ 成功复制: 01492.png
✅ 成功复制: 01740.png
✅ 成功复制: 01989.png
✅ 成功复制: 02237.png
✅ 成功复制: 02486.png
✅ 成功复制: 02734.png
✅ 成功复制: 02983.png
✅ 成功复制: 03231.png
✅ 成功复制: 03479.png
✅ 成功复制: 03728.png
✅ 成功复制: 03976.png
✅ 成功复制: 04225.png
✅ 成功复制: 04473.png
✅ 成功复制: 04722.png
✅ 成功复制: 04970.png
✅ 成功复制: 05219.png
✅ 成功复制: 05467.png
✅ 成功复制: 05715.png
✅ 成功复制: 05964.png
✅ 成功复制: 06212.png
✅ 成功复制: 06461.png
✅ 成功复制: 06709.png
✅ 成功复制: 06958.png
✅ 成功复制: 07206.png
✅ 成功复制: 07455.png
✅ 成功复制: 07703.png
✅ 成功复制: 07952.png
✅ 成功复制: 08200.png
✅ 成功复制: 08449.png
✅ 成功复制: 08697.png
✅ 成功复制: 08946.png
✅ 成功复制: 09194.png
✅ 成功复制: 09443.png
✅ 成功复制: 09691.png
✅ 成功复制: 09940.png
✅ 成功复制: 10188.png
✅ 成功复制: 10437.png
✅ 成功复制: 10685.png
✅ 成功复制: 10933.png
✅ 成功复制: 11182.png
✅ 成功复制: 11430.png
✅ 成功复制: 11679.png
✅ 成功复制: 11927.png
✅ 成功复制: 12176.png
✅ 成功复制: 12

### 每隔10张提取一张

In [2]:
import os
import shutil

# --- 请在这里配置您的文件夹路径和参数 ---

# 1. 源文件夹路径 (包含 01.png - 2422.png 的文件夹)
# 示例: "C:/Users/YourUser/Desktop/SourceImages"
source_folder = "/home/darwin/projects/myNootbook/ToolsBook/output/矿下纹理"

# 2. 目标文件夹路径 (用于保存提取出的图片)
# 示例: "C:/Users/YourUser/Desktop/ExtractedImages"
destination_folder = "/home/darwin/projects/myNootbook/ToolsBook/output/矿下纹理_每隔10张提取"

# 3. 文件名参数
start_num = 1      # 起始编号
end_num = 2422     # 结束编号
interval = 10      # 提取间隔 (每隔 10 张)
file_prefix = ""   # 文件名前缀 (如果没有，保持为空字符串 "")
# ---------------------------------------------
# 文件名格式化说明：
# 假设文件名为 "0001.png", "0011.png" 等，是4位数字
# 那么格式化字符串为 f"{file_prefix}{i:04d}.png"
# 如果是 "01.png", "11.png"，则是 f"{file_prefix}{i:02d}.png"
# 如果是 "1.png", "11.png"，没有补零，则是 f"{file_prefix}{i}.png"
# --- 请根据您的实际文件名修改下面的格式 ---
def get_filename(i):
    # 默认按4位数字补零，例如：0001.png, 0011.png ...
    # 如果您的文件名是 01.png, 11.png 这种，请改成 {i:02d}
    return f"{file_prefix}{i:04d}.png"

# --- 主程序逻辑，一般无需修改 ---

def main():
    """
    主函数，执行图片提取操作
    """
    print("🚀 脚本开始执行...")

    # 1. 检查源文件夹是否存在
    if not os.path.isdir(source_folder):
        print(f"❌ 错误：源文件夹 '{source_folder}' 不存在或不是一个目录。")
        return

    # 2. 创建目标文件夹 (如果不存在)
    os.makedirs(destination_folder, exist_ok=True)
    print(f"📂 目标文件夹 '{destination_folder}' 已准备就绪。")

    # 3. 循环提取文件
    copied_count = 0
    # 使用 range 函数，步长为 interval
    for i in range(start_num, end_num + 1, interval):
        # 构建完整的文件名和路径
        filename = get_filename(i)
        source_path = os.path.join(source_folder, filename)
        destination_path = os.path.join(destination_folder, filename)

        # 检查源文件是否存在
        if os.path.exists(source_path):
            try:
                # 复制文件
                shutil.copy2(source_path, destination_path)
                print(f"✅ 已复制: {filename}")
                copied_count += 1
            except Exception as e:
                print(f"❌ 复制文件 {filename} 时出错: {e}")
        else:
            # 如果文件不存在，打印提示信息
            print(f"🤔 未找到文件: {filename}，已跳过。")

    print("\n🎉 任务完成！")
    print(f"总共提取了 {copied_count} 张图片。")


if __name__ == "__main__":
    main()

🚀 脚本开始执行...
📂 目标文件夹 '/home/darwin/projects/myNootbook/ToolsBook/output/矿下纹理_每隔10张提取' 已准备就绪。
🤔 未找到文件: 0001.png，已跳过。
🤔 未找到文件: 0011.png，已跳过。
🤔 未找到文件: 0021.png，已跳过。
🤔 未找到文件: 0031.png，已跳过。
🤔 未找到文件: 0041.png，已跳过。
🤔 未找到文件: 0051.png，已跳过。
🤔 未找到文件: 0061.png，已跳过。
🤔 未找到文件: 0071.png，已跳过。
🤔 未找到文件: 0081.png，已跳过。
🤔 未找到文件: 0091.png，已跳过。
🤔 未找到文件: 0101.png，已跳过。
🤔 未找到文件: 0111.png，已跳过。
🤔 未找到文件: 0121.png，已跳过。
🤔 未找到文件: 0131.png，已跳过。
🤔 未找到文件: 0141.png，已跳过。
🤔 未找到文件: 0151.png，已跳过。
🤔 未找到文件: 0161.png，已跳过。
🤔 未找到文件: 0171.png，已跳过。
🤔 未找到文件: 0181.png，已跳过。
🤔 未找到文件: 0191.png，已跳过。
🤔 未找到文件: 0201.png，已跳过。
🤔 未找到文件: 0211.png，已跳过。
🤔 未找到文件: 0221.png，已跳过。
🤔 未找到文件: 0231.png，已跳过。
🤔 未找到文件: 0241.png，已跳过。
🤔 未找到文件: 0251.png，已跳过。
🤔 未找到文件: 0261.png，已跳过。
🤔 未找到文件: 0271.png，已跳过。
🤔 未找到文件: 0281.png，已跳过。
🤔 未找到文件: 0291.png，已跳过。
🤔 未找到文件: 0301.png，已跳过。
🤔 未找到文件: 0311.png，已跳过。
🤔 未找到文件: 0321.png，已跳过。
🤔 未找到文件: 0331.png，已跳过。
🤔 未找到文件: 0341.png，已跳过。
🤔 未找到文件: 0351.png，已跳过。
🤔 未找到文件: 0361.png，已跳过。
🤔 未找到文件: 0371.png，已跳过。
🤔 未找到文件: 0381.png，已跳过。
🤔 未找到文件: 03

### 图像拼接2x4

In [3]:
import os
from PIL import Image

# --- 请在这里配置您的参数 ---

# 1. 包含源图片的文件夹路径
#    请确保这个文件夹里正好有8张图片，脚本会按文件名排序后进行拼接。
# 示例: "C:/Users/YourUser/Desktop/MyImages"
input_folder = "/home/darwin/projects/myNootbook/ToolsBook/output/temp_img"

# 2. 拼接后输出的图片文件名和路径
# 示例: "C:/Users/YourUser/Desktop/stitched_output.png"
output_filename = "/home/darwin/projects/myNootbook/ToolsBook/output/temp_img/stitched_output.png"

# 3. 图片之间的间隔（单位：像素）
gap_size = 20

# 4. 拼接后图片的背景颜色 (R, G, B, A)
#    (255, 255, 255, 255) 是白色
#    (0, 0, 0, 255) 是黑色
background_color = (255, 255, 255, 255)

# --- 主程序逻辑，一般无需修改 ---

def stitch_images():
    """
    拼接图片的主函数
    """
    print("🚀 脚本开始执行...")

    # 获取文件夹中所有的图片文件
    try:
        # 支持多种常见图片格式
        supported_formats = ('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff')
        image_files = sorted([f for f in os.listdir(input_folder) if f.lower().endswith(supported_formats)])
    except FileNotFoundError:
        print(f"❌ 错误：输入文件夹 '{input_folder}' 不存在。")
        return

    if len(image_files) < 8:
        print(f"❌ 错误：文件夹中图片少于8张，请检查路径 '{input_folder}'。")
        return
    
    # 我们只处理前8张图片
    images_to_stitch = image_files[:8]
    print(f"🔍 找到并准备拼接以下8张图片: {images_to_stitch}")

    # --- 假设所有图片尺寸相同 ---
    # 打开第一张图片以获取尺寸
    try:
        with Image.open(os.path.join(input_folder, images_to_stitch[0])) as img:
            img_width, img_height = img.size
    except Exception as e:
        print(f"❌ 错误：无法打开第一张图片 '{images_to_stitch[0]}'. 错误信息: {e}")
        return

    # 定义布局
    rows = 2
    cols = 4

    # 计算拼接后大图的总宽度和总高度
    total_width = (cols * img_width) + ((cols - 1) * gap_size)
    total_height = (rows * img_height) + ((rows - 1) * gap_size)

    # 创建一个新的空白图片（画布），用于粘贴所有小图
    canvas = Image.new('RGBA', (total_width, total_height), background_color)
    print(f"🎨 已创建尺寸为 {total_width}x{total_height} 的画布。")

    # 循环遍历图片并粘贴到画布上
    current_x, current_y = 0, 0
    image_index = 0
    for r in range(rows):
        for c in range(cols):
            if image_index < len(images_to_stitch):
                image_path = os.path.join(input_folder, images_to_stitch[image_index])
                try:
                    with Image.open(image_path) as img:
                        # 确保图片尺寸一致，如果不一致则调整为第一张图片的尺寸
                        if img.size != (img_width, img_height):
                            print(f"⚠️ 警告：图片 '{images_to_stitch[image_index]}' 尺寸与第一张不同，将自动调整。")
                            img = img.resize((img_width, img_height))
                        
                        canvas.paste(img, (current_x, current_y))
                        print(f"✅ 已将 '{images_to_stitch[image_index]}' 粘贴到位置 ({current_x}, {current_y})")
                except Exception as e:
                    print(f"❌ 错误：处理图片 '{images_to_stitch[image_index]}' 时出错: {e}")
                
                # 更新下一个图片的x坐标
                current_x += img_width + gap_size
                image_index += 1

        # 一行粘贴完成后，重置x坐标并更新y坐标
        current_x = 0
        current_y += img_height + gap_size

    # 保存最终拼接的图片
    try:
        canvas.save(output_filename)
        print(f"\n🎉 拼接成功！图片已保存为: {output_filename}")
    except Exception as e:
        print(f"❌ 错误：保存最终图片时失败。错误信息: {e}")


if __name__ == "__main__":
    stitch_images()

🚀 脚本开始执行...
🔍 找到并准备拼接以下8张图片: ['1221.png', '1331.png', '1461.png', '1571.png', '1681.png', '1841.png', '2031.png', '2421.png']
🎨 已创建尺寸为 4380x2180 的画布。
✅ 已将 '1221.png' 粘贴到位置 (0, 0)
✅ 已将 '1331.png' 粘贴到位置 (1100, 0)
✅ 已将 '1461.png' 粘贴到位置 (2200, 0)
✅ 已将 '1571.png' 粘贴到位置 (3300, 0)
✅ 已将 '1681.png' 粘贴到位置 (0, 1100)
✅ 已将 '1841.png' 粘贴到位置 (1100, 1100)
✅ 已将 '2031.png' 粘贴到位置 (2200, 1100)
✅ 已将 '2421.png' 粘贴到位置 (3300, 1100)

🎉 拼接成功！图片已保存为: /home/darwin/projects/myNootbook/ToolsBook/output/temp_img/stitched_output.png
